In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import pickle
import random

In [48]:
model_name = "allenai/unifiedqa-v2-t5-base-1251000"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
with open("./data/constraints_v2.json", 'r') as f:
    constraint_data = json.load(f)

In [12]:
from belief.evaluation import calibrate, build_bb, load_facts
from belief.lmbb import LMBB, FeedbackType

In [4]:
fact_batches = load_facts("./data/calibration_facts.json", num_batches=10)

In [76]:
lmbb = LMBB(model=model, tokenizer=tokenizer, raw_constraints=constraint_data['links'], forward_weight=10, backward_weight=0.7)
# lmbb.DEBUG = True

In [64]:
i = 0

In [ ]:
lmbb.add_belief_set(fact_batches[i])
print(i, lmbb.calculate_f1(sum(fact_batches[:i+1], [])), lmbb.calculate_consistency())
i += 1

In [26]:
i = 4

In [66]:
weights = [0.1, 0.3, 0.7, 1, 3, 7, 10]

In [11]:
_ = build_bb(
    model_name="allenai/unifiedqa-v2-t5-base-1251000",
    facts_file="./data/silver_facts.json", 
    constraints_file="./data/constraints_v2.json",
    num_batches=10,
    constraint_solving=True,
    with_feedback=True,
    forward_weight=6, 
    backward_weight=0.6
)

del _

Batch 1 : F1 = 0.6399197547963046, consistency = 1.0
Batch 2 : F1 = 0.6595744634460446, consistency = 1.0
Batch 3 : F1 = 0.6783369755720908, consistency = 1.0
Batch 4 : F1 = 0.7023945219955562, consistency = 1.0
Batch 5 : F1 = 0.7300236358041547, consistency = 1.0
Batch 6 : F1 = 0.746378285249434, consistency = 1.0
Batch 7 : F1 = 0.7722091501193691, consistency = 1.0
Batch 8 : F1 = 0.7957267674519273, consistency = 1.0
Batch 9 : F1 = 0.8166888543126881, consistency = 1.0
Batch 10 : F1 = 0.8391877008198719, consistency = 1.0


In [17]:
f1s = [
    0.6399197547963046, 
    0.6595744634460446, 
    0.6783369755720908, 
    0.7023945219955562,
    0.7300236358041547, 
    0.746378285249434, 
    0.7722091501193691,
    0.7957267674519273,
    0.8166888543126881,
    0.8391877008198719
]

consistencies = [1.0 for _ in range(10)]

d = dict(
    model_name="allenai/unifiedqa-v2-t5-base-1251000",
    facts_file="./data/silver_facts.json",
    constraints_file="./data/constraints_v2.json",
    num_batches=10,
    constraint_solving=True,
    with_feedback=True,
    feedback_type=FeedbackType.RELEVANT,
    forward_weight=6,
    backward_weight=0.6,
    f1s=f1s,
    consistencies=consistencies,
    final_f1=f1s[-1],
    final_consistency=consistencies[-1]
)

In [28]:
with open('./results/results.pkl', 'rb') as f:
    results = pickle.load(f)
results.append(d)
with open('./results/results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [29]:
with open('./results/results.pkl', 'rb') as f:
    results = pickle.load(f)

In [31]:
_ = build_bb(
    model_name="allenai/unifiedqa-v2-t5-base-1251000",
    facts_file="./data/silver_facts.json", 
    constraints_file="./data/constraints_v2.json",
    num_batches=10,
    constraint_solving=False,
    with_feedback=False,
    forward_weight=6, 
    backward_weight=0.6
)

del _

Batch 1 : F1 = 0.596273287573404, consistency = 0.8038461538461539
Batch 2 : F1 = 0.5994694916729912, consistency = 0.8007554296506137
Batch 3 : F1 = 0.6041542983201754, consistency = 0.8075664621676891
Batch 4 : F1 = 0.6090443262191578, consistency = 0.8078233161191077
Batch 5 : F1 = 0.6082142813601136, consistency = 0.8097634972074743
Batch 6 : F1 = 0.6105797924071288, consistency = 0.8101168437755446
Batch 7 : F1 = 0.607506767220549, consistency = 0.8058636524196396
Batch 8 : F1 = 0.6095259448680024, consistency = 0.8050346845870366
Batch 9 : F1 = 0.6086348147538226, consistency = 0.8052200679763152
Batch 10 : F1 = 0.6061863064121384, consistency = 0.805502425029032
